In [12]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
from aesara.tensor.subtensor import set_subtensor

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Wine classification

This example demonstrates ...

Adapted from [unit 10: italywines123.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/italywines123.odc).

Data can be found [here](https://archive.ics.uci.edu/ml/datasets/wine).

## Associated lecture video: Unit 10 Lesson 5

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=102" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

This popular data set was provided by Forina et al (1988). 
The data below consist of results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

  The variables are
  
   Column   Variable   Description  
   
  ==========================
  
     1      Y     Type (Response, 1,2,3)  
                      1 == 1 0 0;    2 == 0 1 0;   3 == 0 0 1
                      
     2      X1    Alcohol 
      
 	 3      X2    Malic acid 
     
 	 4      X3    Ash 
     
	 5      X4    Alcalinity of ash 
      
  	 6      X5    Magnesium 
    
	 7      X6    Total phenols 
      
 	 8      X7    Flavanoids 
     
     9      X8    Nonflavanoid phenols 
      
     10     X9    Proanthocyanins 
     
     11     X10  Color intensity 
     
     12     X11  Hue 
     
     13     X12 OD280/OD315 of diluted wines 
     
     14     X13  Proline 
 



(a) Fit the multinomial regression
that predicts the type of wine Y from predictors  X1 - X13.
What are estimated coefficients? What is the deviance?

(b) What is your prediction for  pp=P(Ynew=1)  if a new case has attributes  
 XX1 = 12.9, XX2 = 2,  XX3 = 2.4, XX4 = 17,
 XX5 = 100,  XX6 = 2.8, XX7 = 2.1, XX8 =  0.35,  XX9 = 1.6, XX10 = 5, 
XX11 = 1.05,  XX12 = 3, and XX13 = 750?
How would you classify this wine type, as 1, 2, or 3?

 
 Forina, M.,   Leardi, R.,   Armanino, C., and  Lanteri, S. (1988).
 PARVUS: An extendable package of programs for data exploration, classification and correlation,  Elsevier, Amsterdam,   ISBN 0-444-43012-1;
 also, Report at Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno,
  16147 Genoa, Italy.

In [13]:
data = pd.read_csv("../data/wine.data", header=None)
Y = pd.get_dummies(data[0]).to_numpy()
X = data.drop(0, axis=1).to_numpy()
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [26]:
with pm.Model() as m:
    _b = pm.Normal("_b", 0, tau=0.05, shape=(14, 3))
    b = pm.Deterministic("b", set_subtensor(_b[:, 0], 0))
    phi = pm.math.dot(X_aug, b)
    P = pm.math.exp(phi / pm.math.sum(phi, axis=1))

    pm.Multinomial("likelihood", n=1, p=P, observed=Y)
    
    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


ValueError: Input dimension mismatch. One other input has shape[1] = 3, but input[1].shape[1] = 178.
Apply node that caused the error: Elemwise{TrueDiv}[(0, 0)](Dot22.0, InplaceDimShuffle{x,0}.0)
Toposort index: 6
Inputs types: [TensorType(float64, (None, None)), TensorType(float64, (1, None))]
Inputs shapes: [(178, 3), (1, 178)]
Inputs strides: [(24, 8), (1424, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{exp,no_inplace}(Elemwise{TrueDiv}[(0, 0)].0), Elemwise{Composite{Switch(EQ(i0, i1), i2, (i3 * i4))}}[(0, 0)](Elemwise{exp,no_inplace}.0, TensorConstant{(1, 1) of 0}, TensorConstant{[[-inf   0.. 0. -inf]]}, TensorConstant{[[1. 0. 0...0. 0. 1.]]}, Elemwise{TrueDiv}[(0, 0)].0)]]

HINT: Re-running with most Aesara optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the Aesara flag 'optimizer=fast_compile'. If that does not work, Aesara optimizations can be disabled with 'optimizer=None'.
HINT: Use 

In [ ]:
az.summary(trace)